# Dian 24 秋招 TEST 2

## 绝对位置编码

In [2]:
import torch
import torch.nn as nn
import math
import numpy as np
import pandas as pd

In [3]:
class AbsolutePositionalEncoding(nn.Module):
    def __init__(self, max_positions, feature_dimensions):
        super().__init__()
        # 初始化位置编码张量
        positional_encoding = torch.zeros(max_positions, feature_dimensions)
        position_indices = torch.arange(0, max_positions).unsqueeze(1).float()

        # 计算频率项
        frequency_terms = torch.arange(0, feature_dimensions, 2).float()
        frequency_terms = torch.exp(frequency_terms * (-math.log(10000.0) / feature_dimensions))

        # 使用正弦和余弦函数填充位置编码
        positional_encoding[:, 0::2] = torch.sin(position_indices * frequency_terms)
        positional_encoding[:, 1::2] = torch.cos(position_indices * frequency_terms)

        # 注册为缓冲区
        self.register_buffer('positional_encoding', positional_encoding.unsqueeze(0))

    def forward(self, x):
        # 将位置编码添加到输入上
        return x + self.positional_encoding[:, :x.size(1)].clone().detach()

In [10]:
max_pos=10
feature_dims=4

model=AbsolutePositionalEncoding(max_pos,feature_dims)

x=torch.normal(0,1,(3,6,feature_dims))

output=model(x)

print(f'输入:{x}')
print(f'绝对位置编码:{model.positional_encoding}')
print(f'最终输出：{output}')

输入:tensor([[[ 0.6338, -1.9645,  0.4607,  0.4603],
         [-1.0848,  0.7344, -1.3640,  2.5334],
         [ 0.6434,  0.7654, -0.2773, -0.1734],
         [-2.2900,  2.3737, -0.0915, -1.3662],
         [ 0.2725, -0.6552,  0.5157,  0.6238],
         [ 0.3674,  1.1737, -1.2677, -0.8064]],

        [[ 0.9800, -0.3986,  0.3927,  2.1708],
         [ 1.6249, -1.5686,  0.2860, -0.3246],
         [-0.3159,  1.6060, -0.6613, -0.6006],
         [-0.2555,  0.1319, -0.8683, -0.2944],
         [-0.6719,  1.1667, -0.1266,  0.5282],
         [-0.3447,  0.1421, -0.3103,  0.8050]],

        [[ 0.5169, -0.8885, -0.8847, -1.2259],
         [-0.2113, -0.3036,  0.6646,  0.6098],
         [-1.9815,  0.5544, -0.7298,  0.6059],
         [-0.9416, -0.2611, -1.1957,  1.6762],
         [-1.7945, -0.7496,  0.2069, -1.0592],
         [ 0.6726, -0.8612, -0.0480, -0.5065]]])
绝对位置编码:tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.0100,  0.9999],
         [ 0.9093, -0.4161,  0.0200,  0.9998

## 旋转位置编码

In [39]:
import torch
import torch.nn as nn
import math

class RotaryPosEncoding(nn.Module):
    def __init__(self, max_pos, feature_dims):
        super().__init__()
        self.max_pos = max_pos
        self.feature_dims = feature_dims

        # 创建旋转位置编码矩阵
        self.create_rotary_pos_encoding()

    def create_rotary_pos_encoding(self):
        # 位置张量
        position = torch.arange(0, self.max_pos).unsqueeze(1).float()
        print(position.shape)
        # 维度张量
        dim = torch.arange(0, self.feature_dims // 2).float()
        print(f'dims:{dim.shape}')
        
        # 计算角度
        angles = position*torch.exp(-math.log(10000.0) * dim / (self.feature_dims // 2))#theta
        angles = angles.unsqueeze(2)  # (max_pos, feature_dims / 2, 1)
        print(f'angles:{angles.shape}')

        # 计算旋转的正弦和余弦
        self.sin_angles = torch.sin(angles)
        print(self.sin_angles.shape)
        self.cos_angles = torch.cos(angles)
    
    def get_pos_encoding(self, seq_len):
        # 获取前seq_len个位置编码
        # 计算每个位置的sin和cos
        sin_pos = self.sin_angles[:seq_len, :, 0]  # (seq_len, feature_dims / 2)
        print(sin_pos.shape)
        cos_pos = self.cos_angles[:seq_len, :, 0]  # (seq_len, feature_dims / 2)
        
        # 组合sin和cos成完整的编码，将两者分别对自己concat,实现重复效果
        cos_pos_encoding = torch.cat([cos_pos, cos_pos], dim=1)
        print(f'a{cos_pos_encoding.shape}')
        cos=cos_pos_encoding[6,:]
        print(f's:{cos.shape}')
        cos=cos.numpy()
        cos=pd.DataFrame(cos) 
        cos.to_excel('cos_pos_encoding.xlsx') 


        sin_pos_encoding = torch.cat([sin_pos, sin_pos], dim=1)
        sin=sin_pos_encoding[6,:]
        print(f's:{sin.shape}')
        sin=sin.numpy()
        sin=pd.DataFrame(sin) 
        sin.to_excel('sin_pos_encoding.xlsx')

        print(f'11{cos_pos_encoding.shape}')
        print(f'11{sin_pos_encoding.shape}')

        return cos_pos_encoding.unsqueeze(0),cos_pos_encoding.unsqueeze(0) # (1, seq_len, feature_dims)


    def forward(self, x):#自动执行
        # x的形状是 (batch_size, seq_len, feature_dims)
        batch_size, seq_len, feature_dims = x.size()
        
        # 生成正弦和余弦位置编码
        cos_pos_encoding,sin_pos_encoding = self.get_pos_encoding(seq_len)
        
        # 将位置编码应用到x上
        x = x * cos_pos_encoding+x*sin_pos_encoding
        return x

    
# 示例用法
max_pos = 512
feature_dims = 64
x = torch.randn(32, 50, feature_dims)  # (batch_size, seq_len, feature_dims)

rop = RotaryPosEncoding(max_pos, feature_dims)
encoded_x = rop(x)
print(encoded_x.shape)  
print(encoded_x)


torch.Size([512, 1])
dims:torch.Size([32])
angles:torch.Size([512, 32, 1])
torch.Size([512, 32, 1])
torch.Size([50, 32])
atorch.Size([50, 64])
s:torch.Size([64])
s:torch.Size([64])
11torch.Size([50, 64])
11torch.Size([50, 64])
torch.Size([32, 50, 64])
tensor([[[ 3.3703, -0.0894,  1.5173,  ..., -2.4980, -1.1261, -0.9834],
         [ 0.7898,  1.0664, -1.6102,  ..., -1.1337,  0.7493,  1.9031],
         [ 0.0513, -0.0816, -0.9557,  ..., -1.2478,  0.6133,  1.2436],
         ...,
         [-2.2580,  0.6525, -0.6725,  ..., -1.9854, -1.2191,  3.9543],
         [-0.9416, -0.4037, -0.4872,  ...,  0.5792,  2.8435,  2.7197],
         [-0.1767,  1.0492, -0.3057,  ...,  1.1203, -0.1899, -0.9896]],

        [[-0.0162, -1.3850, -1.5298,  ...,  0.6609, -1.6867, -0.7495],
         [-0.0843,  0.7929, -0.8734,  ...,  1.3991, -0.3363, -3.1567],
         [-0.9206,  0.1070,  0.4149,  ..., -0.7691,  2.5169, -0.6010],
         ...,
         [ 2.1724,  1.3049, -0.1233,  ...,  2.7875, -3.4522,  2.0544],
        